# 线性回归
> 合抱之木，生于毫末；九层之台，起于累土；千里之行，始于足下。
>
> 《道德经：第六十四章》

伟大的成就往往始于一个简单的想法。

**深度学习**（Deep Learning）也不例外。它起源于1943年一个极其朴素的数学模型：**人工神经元**（Artificial Neuron）。

---

人类的大脑并非一个整体的超级处理器，而是由大约 860 亿个神经元组成的复杂网络。单个神经元的能力非常有限：

1. 接收信号；
2. 判断信号强度；
3. 决定是否通知下一个神经元。

这种机制可以概括为四个步骤：**输入**、**加权**、**过滤**和**输出**。

受此启发，神经科学家麦卡洛克（McCulloch）与逻辑学家皮茨（Pitts）提出了著名的 **M-P 神经元模型**：

$$
\text{output} =
\begin{cases}
1 & \text{if } \sum\limits_{i=1}^{n} w_i x_i \geq \theta \\
0 & \text{otherwise}
\end{cases}
$$

该模型包含两个核心思想：

1. **线性回归**（Linear Regression）：对输入进行加权求和；
2. **激活函数**（Activation Function）：通过阈值 $\theta$ 进行信号过滤。


``💡 最初的 M-P 模型使用阶跃函数（Step Function），现代深度学习改为"连续可导"的激活函数。``

```{figure} images/neuron.png
:align: center
:width: 480px

**图例：人工神经元结构**

* $x_1$、$x_2$：（输入）特征值；
* $w_1$、$w_2$：权重；
* $p$：（输出）预测值。

```

---

单个神经元能力有限，但当我们将成百上千，甚至数十亿个神经元连接并堆叠成多层结构时，就形成了功能强大的**深度神经网络**（Deep Neural Networks）。

```{figure} images/deep-neural-network.png
:align: center
:width: 400px

**图例：（两层）神经网络结构**

* $x_1$、$x_2$：（输入）特征值；
* $H_1$、$H_2$、$H_3$、$H_4$：第一层神经元；
* $h_1$、$h_2$、$h_3$、$h_4$：中间值；
* $O$：第二层神经元；
* $p$：（输出）预测值。

```

---

现在，我们就从最简单的单神经元模型开始，逐步构建出能理解语言，生成内容的人工智能系统。

## 最简单的神经网络模型

一个仅含单个神经元的网络模型，虽然结构简单，却已经能完成许多实用任务。例如：

* 趋势预测：如房价、股票或者销量预测；
* 影响分析：如健康风险评估、营销回报归因。

---

我们以一个冰激凌销量预测问题为例：

> 小明经营一家冰激凌店，发现销量与天气密切相关。他希望构建一个网络模型，根据每天的温度和湿度预测当天的冰激凌销量。

In [1]:
import numpy as np

``💡 NumPy 是 Python 的核心科学计算库，提供高效的多维数组（ndarray）和向量化运算。本书将全程使用 NumPy 实现所有计算。``

## 数据

单个神经元功能非常简单，就是完成一次加权求和（线性回归）：

$$
p = \sum_{i=1}^{n} w_i x_i + b
$$

这里：
* $x_i$：第 $i$ 个输入数据，也称为**特征值**（Feature）；
* $w_i$：对应的**权重**（Weight）；
* $b$：**偏置**（Bias）；
* $p$：加权求和的计算结果，也称为**预测值**（Prediction）。

``💡 完整的神经元模型包括线性回归和激活函数两个部分。但是在实践中，激活函数会被单独实现，并和神经元配合使用。``

### 特征

我们用一个 NumPy 数组保存今天的天气情况：[ 温度, 湿度 ]。

小明找来了今天的天气预报，于是我们有了第一组输入数据，包括两个特征值。

In [2]:
feature = np.array([28.1, 58.0])

## 模型

**模型**（Model）包括完成预测所需的全部参数（权重和偏置）和计算逻辑。

### 参数：权重、偏置

权重的数值和特征值的数值一一对应。

比如：特征值 $[x_1, x_2]$ 包括两个数值：第一个是温度，第二个是湿度。权重也必须是两个数值：$[w_1, w_2]$。

权重代表的是每个特征值对预测结果的影响力。

比如：如果 $w_1$ 很大， $w_2$ 很小，就说明大家主要因为天气太热而购买冰激凌，而不是因为空气潮湿而购买冰激凌。

偏置只有一个数值，代表的是不受特征值影响的基础量。

这可能是来自一部分冰激凌爱好者的销量。他们无论天气热不热、还是空气潮湿不潮湿，都会买冰激凌。

---

我们同样用 NumPy 的数组来保存参数。

作为初始值，我们使用 NumPy 的函数生成权重和偏置：

* 权重：二维数组，每行代表一个神经元的权重。
* 偏置：一维数组，每个数值代表一个神经元的偏置。

In [3]:
weight = np.ones((1, 2)) / 2    # [[0.5, 0.5]]
bias = np.zeros(1)              # [0.]

``💡 作为初始值，我们并不确定温度和湿度哪个对冰激凌销量影响更大。所以假设它们一样重要，分别给 0.5 的权重；而将偏置设为 0，是因为我们并不知道会有多少来自冰激凌爱好者的稳定销量。``

### 推理函数

推理函数实现加权求和（线性回归）的功能。这种数据从输入向输出方向的流动，称为**前向传播**（Forward Propagation）。

我们利用 NumPy 提供的矩阵运算（比如：矩阵转置、矩阵乘法），可以高效地实现推理函数：

$$
p = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \cdot \begin{bmatrix} w_1 \\ w_2 \end{bmatrix} + b
$$

计算结果是一个只有一个数值的数组：$[p_1]$，也就是我们希望获得的预测值。

In [4]:
def forward(x, w, b):
    return x @ w.T + b

## 验证

短短几行代码，我们已经构建出一个可运行的神经网络！

### 推理

运行一个网络模型来进行预测的过程，被称为**模型推理**（Model Inference）。

现在，让我们的网络模型帮小明预测一下今天的冰激凌销量。

In [5]:
prediction = forward(feature, weight, bias)
print(f'prediction: {prediction}')

prediction: [43.05]


模型预测：今天小明大概可以卖出 43 个冰激凌。

这个预测靠谱吗？等到晚上我们就知道结果了。

## 课后练习

如果想添加更多的特征值（比如：风速），应该怎么调整我们的模型？